In [1]:
import torch
import torch.nn as nn
from rfft_lib import rfft_audio
import onnx

In [2]:
audio_path = "/home/satya-tt0492/Documents/TT/ex-7/audio/small.wav"

In [3]:
audio = rfft_audio.load_audio(audio_path)
print(audio)

tensor([ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ..., -6.1035e-05,
         3.0518e-05,  0.0000e+00])


In [4]:
audio.shape

torch.Size([262094])

In [5]:
rfft_audio.apply_rfft(audio).shape

torch.Size([131048])

In [6]:
class RFFTModel(nn.Module):
    def __init__(self):
        super(RFFTModel,self).__init__()
    
    def forward(self, audio):
        rfft_magnitude = rfft_audio.apply_rfft(audio)
        return rfft_magnitude

In [7]:
model = RFFTModel()

In [8]:

export_option = torch.onnx.ExportOptions(dynamic_shapes=True)
onnx_program = torch.onnx.dynamo_export(
    model,
    audio,
    # export_options = export_option
)

/home/satya-tt0492/anaconda3/envs/ex5/lib/python3.9/site-packages/onnxscript/converter.py:820: FutureWarning: 'onnxscript.values.Op.param_schemas' is deprecated in version 0.1 and will be removed in the future. Please use '.op_signature' instead.
  param_schemas = callee.param_schemas()
/home/satya-tt0492/anaconda3/envs/ex5/lib/python3.9/site-packages/onnxscript/converter.py:820: FutureWarning: 'onnxscript.values.OnnxFunction.param_schemas' is deprecated in version 0.1 and will be removed in the future. Please use '.op_signature' instead.
  param_schemas = callee.param_schemas()
/home/satya-tt0492/anaconda3/envs/ex5/lib/python3.9/site-packages/torch/onnx/_internal/exporter.py:137: UserWarning: torch.onnx.dynamo_export only implements opset version 18 for now. If you need to use a different opset version, please register them with register_custom_op.
  warnings.warn(
/home/satya-tt0492/anaconda3/envs/ex5/lib/python3.9/site-packages/torch/onnx/_internal/fx/onnxfunction_dispatcher.py:529:

In [9]:
onnx_program.save("rfft_model.onnx")

In [10]:
onnx_model = onnx.load("rfft_model.onnx")
onnx.checker.check_model(onnx_model)

In [11]:
import onnxruntime

In [12]:
ort_session = onnxruntime.InferenceSession("rfft_model.onnx")

In [13]:
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

In [14]:
onnx_input = onnx_program.adapt_torch_inputs_to_onnx(audio)

In [15]:
onnxruntime_input = {k.name: to_numpy(v) for k, v in zip(ort_session.get_inputs(), onnx_input)}
onnx_output = ort_session.run(None,onnxruntime_input)

In [16]:
onnx_output[0]

array([2.2597394 , 2.1534455 , 2.2468333 , ..., 0.08980468, 0.22606245,
       0.22876866], dtype=float32)

In [17]:
rfft_direct = rfft_audio.apply_rfft(audio).numpy()

In [18]:
rfft_direct[:10]

array([2.2597651, 2.1731648, 2.2680433, 2.1840084, 2.1720464, 2.177937 ,
       2.170428 , 2.2503257, 2.2100704, 2.3014824], dtype=float32)

In [19]:
onnx_output[0][:10]

array([2.2597394, 2.1534455, 2.2468333, 2.1685116, 2.1478982, 2.1269329,
       2.1952722, 2.3304734, 2.2769892, 2.4249902], dtype=float32)

In [20]:
import numpy as np

In [21]:
np.mean(np.abs(onnx_output[0] - rfft_direct))

0.27339584